In [1]:
import pandas as pd
import os
import json

In [2]:
metrics_dir = 'calculated/'
courses = []

for file in os.listdir(metrics_dir):
    course_name = os.fsdecode(file)
    if '.txt' not in course_name:
        courses.append(pd.read_csv(metrics_dir+course_name).set_index('element_id'))
df = pd.concat(courses)

In [3]:
df['skips_percentage'] /= 100
df['lost_percentage'] /= 100
df['solved_percentage'] /= 100
df['guessed_percentage'] /= 100
df['N_tries'] /= 100
df

,mean_tries_count,skips_percentage,lost_percentage,guessed_percentage,solved_percentage,mean_time,N_tries
element_id,,,,,,,
293,1.947368,0.051813,0.093596,0.013158,0.986842,1.378641,0.01
298,1.603175,0.004695,0.072727,0.046154,0.984615,1.209591,0.01
299,1.000000,0.130841,0.276382,0.000000,1.000000,0.000000,0.00
300,1.000000,0.123853,0.055215,0.000000,1.000000,0.000000,0.00
301,1.000000,0.018100,0.051613,0.000000,1.000000,0.000000,0.00
...,...,...,...,...,...,...,...
9396,1.685410,0.053122,NaN,0.000000,0.966565,2.441592,0.00
9597,2.093355,0.177552,NaN,0.000000,0.938088,2.527137,0.02
10959,1.586679,0.110403,0.407436,0.000000,0.953467,1.755202,0.00


In [4]:
dict = {}
for metric in df.columns:
    dict[metric] = [-1]*5
    for i in range(95, 100):
        quantile = i
        if metric == 'solved_percentage':
            quantile = 100 - i
        threshold = df[metric].quantile(quantile/100)
        dict[metric][min(quantile, 100-quantile)-1] = threshold

with open('super_thresholds.json', 'w') as fp:
    json.dump(dict, fp)

In [5]:
with open('super_thresholds.json', 'r') as fp:
    super_thresholds = json.load(fp)

def super_thresholds_hint(df: pd.DataFrame):
    """df - DataFrame с метриками одного модуля"""
    for idx, task in df.iterrows():
        hard_messages = []
        for metric in task.index:
            top_percent = None
            for i in range(4, -1, -1):
                if (metric != 'solved_percentage') & (df.loc[idx, metric] != pd.NA) & (df.loc[idx, metric] < super_thresholds[metric][i]):
                    break
                if (metric == 'solved_percentage') & (df.loc[idx, metric] > super_thresholds[metric][i]):
                    break    
                top_percent = i+1
            if top_percent != None:
                hard_messages.append(f'топ {top_percent}% задач по {metric}')
        if hard_messages:
            message = ', '.join(hard_messages)
            print(f'Задача {idx} входит в {message}.')

In [6]:
super_thresholds_hint(df[:60])

Задача 318 входит в топ 4% задач по guessed_percentage, топ 3% задач по solved_percentage.
Задача 319 входит в топ 4% задач по lost_percentage.
Задача 330 входит в топ 1% задач по lost_percentage.
Задача 332 входит в топ 1% задач по lost_percentage.
Задача 337 входит в топ 4% задач по lost_percentage.
Задача 342 входит в топ 5% задач по lost_percentage.
Задача 354 входит в топ 1% задач по lost_percentage.
Задача 361 входит в топ 3% задач по mean_tries_count, топ 2% задач по lost_percentage, топ 2% задач по guessed_percentage, топ 1% задач по solved_percentage, топ 1% задач по mean_time.
Задача 362 входит в топ 4% задач по guessed_percentage, топ 1% задач по solved_percentage, топ 5% задач по mean_time.
